<a href="https://colab.research.google.com/github/RIPS-2024-Aerospace/Aerospace-Project/blob/main/BhatDist_BFGS_LTTOptimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!git clone https://github.com/RIPS-2024-Aerospace/Aerospace-Project.git

Cloning into 'Aerospace-Project'...
remote: Enumerating objects: 489, done.
remote: Counting objects: 100% (220/220), done.
remote: Compressing objects: 100% (159/159), done.
remote: Total 489 (delta 143), reused 93 (delta 61), pack-reused 269
Receiving objects: 100% (489/489), 31.46 MiB | 22.05 MiB/s, done.
Resolving deltas: 100% (233/233), done.


In [40]:
import numpy as np
import random
import math
import scipy as sp

np.random.seed(29)

%run "/content/Aerospace-Project/DiffusionLunarKF.ipynb"
%run "/content/Aerospace-Project/CentralizedLunarKF.ipynb"
%run "/content/Aerospace-Project/FilterComparison.ipynb"
%run "/content/Aerospace-Project/LTT_GetBhatDistance.ipynb"

fatal: destination path 'Aerospace-Project' already exists and is not an empty directory.


In [29]:
#Generates a random edge weighting for Lunar network
#Output is vector of size 8, weights correspond to edges: [(1,2), (1,3), (1,4), (1,5), (2,1), (3,1), (4,1), (5,1)]
def gen_first_guess():
  weights = []
  temp = []
  for i in range(5):
    temp.append(random.random())
  weights = [w/sum(temp) for w in [temp[i] for i in range(4)]]
  for j in range(4):
    edge_node = [random.random()]
    weights += edge_node
  return weights

In [30]:
#Converts len=8 vector to full adjacency matrix for use in filtering
def convert_to_adj(weights):
  network = []
  temp = []
  for i in range(4):
    temp.append(weights[i])
  network.append(1-sum(temp))
  network += temp
  for i in range(4, len(weights)):
    network.append(weights[i])
    network.append(1-weights[i])



  ref = np.array([[0.2,0.2,0.2,0.2,0.2], [0.5, 0.5, 0, 0,0], [0.5, 0, 0.5, 0,0], [0.5,0,0, 0.5,0],[0.5,0,0,0,0.5]]) #This just serves as reference for where there are edges
  W = []
  i = 0
  for arr in ref:
    for val in arr:
      if val != 0:
        W.append(network[i])
        i += 1
      else: W.append(0)
  adj_matrix = np.array([[W[i] for i in range(0,5)], [W[i] for i in range(5,10)], [W[i] for i in range(10,15)], [W[i] for i in range(15,20)], [W[i] for i in range(20,25)]])
  return(adj_matrix)

In [46]:
#Cost function to use in BFGS (Output = Bhattacharya Distance)
def cost_function(weights):
  C = convert_to_adj(weights)
  return(run_everything(C))

In [54]:
#Run BFGS for LTT
import scipy as sp
import numpy as np

def test_bfgs():

  #Linear Constraints:
  A = np.array([[1,0,0,0,0,0,0,0],
              [0,1,0,0,0,0,0,0],
              [0,0,1,0,0,0,0,0],
              [0,0,0,1,0,0,0,0],
              [1,1,1,1,0,0,0,0],
              [0,0,0,0,1,0,0,0],
              [0,0,0,0,1,0,0,0],
              [0,0,0,0,0,1,0,0],
              [0,0,0,0,0,1,0,0],
              [0,0,0,0,0,0,1,0],
              [0,0,0,0,0,0,1,0],
              [0,0,0,0,0,0,0,1],
              [0,0,0,0,0,0,0,1]])
  cons = sp.optimize.LinearConstraint(A, lb = np.zeros(13), ub = np.ones(13))
  bounds = sp.optimize.Bounds(lb = np.zeros(8), ub = np.ones(8))

  #Initial Guess:
  x0 = gen_first_guess()

  result = sp.optimize.minimize(fun=cost_function, x0=x0, bounds=bounds, constraints=cons)

  print("Optimized Weights Vector: ", np.around(result.x, 4))
  print("\nCorresponding Adjacency Matrix:")
  print(convert_to_adj(np.around(result.x, 4)))
  print("\nResulting Bhattacharya Distance: ", round(result.fun, 4))


test_bfgs()

Optimized Weights Vector:  [0.7461 0.0497 0.0441 0.0893 0.4247 0.5756 0.8733 0.9316]

Corresponding Adjacency Matrix:
[[0.0708 0.7461 0.0497 0.0441 0.0893]
 [0.4247 0.5753 0.     0.     0.    ]
 [0.5756 0.     0.4244 0.     0.    ]
 [0.8733 0.     0.     0.1267 0.    ]
 [0.9316 0.     0.     0.     0.0684]]

Resulting Bhattacharya Distance:  5.5227
